# Example Notebook for transport measurements 

## starting qkit

In [ ]:
import numpy as np
import time
import qkit

In [ ]:
qkit.cfg['run_id']='run_number'
qkit.cfg['user']='User'
qkit.start()

In [ ]:
from qkit.measure.transport import transport
import qkit.measure.samples_class as sc

## Initialize devices

Source measure unit combined with tunnel electronic as IV device

In [ ]:
SMU = qkit.instruments.create('SMU', 'Keithley', address='TCPIP0::00.00.000.00::INSTR', reset=True)  # Keithley_2636A
#SMU = qkit.instruments.create('SMU', 'Yokogawa', address='TCPIP0::00.00.000.00::INSTR', reset=True)  # Yokogawa_GS820
IVD = qkit.instruments.create('IVD', 'virtual_tunnel_electronic', SMU=SMU)

alternative: Source measure unit as IV device

In [ ]:
IVD = qkit.instruments.create('IVD', 'Keithley', address='TCPIP0::00.00.000.00::INSTR', reset=True)  # Keithley 2636A
#IVD = qkit.instruments.create('IVD', 'Yokogawa', address='TCPIP0::00.00.000.00::INSTR', reset=True)  # Yokogawa GS820

Microwave source

In [ ]:
mw_src = qkit.instruments.create('mw_src','Anritsu_MG37022', address = 'TCPIP0::00.00.000.00::inst0::INSTR')  # Anritsu MG37022

Temperature control

In [ ]:
tip = qkit.instruments.create('tip', 'TIP_client', address='IP address', port='PORT')

## Initialize transport measurements

In [ ]:
reload(transport)
tr = transport.transport(IVD)

In [ ]:
sample = sc.Sample()
sample.name = 'A0000#0'
sample.comment = 'This sample looks promising.'
sample.ports = 'I0to0_V0to0'
print(sample.get_all())

In [ ]:
tr.set_filename(filename='filename')
tr.set_expname(expname='expname')
tr.set_comment(comment='comment')
tr.set_sample(sample)

In [ ]:
IVD.reset()
IVD.set_defaults(sweep_mode=0)  # sweep_mode: 0 (VV-mode) | 1 (IV-mode) | 2 (VI-mode)
IVD.get_all()

# measure 1D scan

IVD settings

In [ ]:
IVD.set_sense_nplc(1)  # number of power line cycles
IVD.set_sense_average(1)  # average
IVD.set_bias_delay(15e-6) # for Yokogawa GS820 it should be >= 200e-6 to ensure correct data storage
IVD.set_sense_delay(15e-6)  # measurement delay

In [ ]:
################
# current bias #
################
IVD.set_sweep_mode(0) # 0 (VV-mode) | 1 (IV-mode) | 2 (VI-mode)
IVD.set_pseudo_bias_mode(0) # 0 (current bias) | 1 (voltage bias)
IVD.set_dAdV(2e-6)  # conversion factor of voltage biased current source (U -> I)
IVD.set_amp(1e3)  # preamplifier factor
IVD.set_BW(100)  # bandwidth of external filter (in Hz)
IVD.set_tau_amp(100)  # time constant of external amplifier (τ=2πRC in s)

In [ ]:
################
# voltage bias #
################
IVD.set_pseudo_bias_mode(1)  # voltage bias instead of IVD.set_pseudo_bias_mode(0) 
IVD.set_Vdiv(1e3)  # external voltage divider factor (e.g. 1e3)
IVD.set_dVdA(1e8)  # amplification factor for voltage bias (e.g. 1e9)

transport settings

In [ ]:
tr.set_dVdI(True)  # data series of numerical gradient
tr.set_average(3)  # average entire traces
tr.sweeps.reset_sweeps()
tr.add_sweep_4quadrants(start=0, stop=100e-9, step=1e-9, offset=0) # 4 quadrants sweep: start at zero -> stop -> zero -> -stop -> zero
#tr.sweeps.add_sweep(start=0, stop=0, step=0) # single sweep

start 1D measurement

In [ ]:
tr.measure_1D()

# 2D scan

IVD settings

In [ ]:
IVD.set_sense_nplc(1)  # number of power line cycles
IVD.set_sense_average(1)  # average
IVD.set_bias_delay(15e-6) # for Yokogawa GS820 it should be >= 200e-6 to ensure correct data storage
IVD.set_sense_delay(15e-6)  # measurement delay

In [ ]:
################
# current bias #
################
IVD.set_sweep_mode(0) # 0 (VV-mode) | 1 (IV-mode) | 2 (VI-mode)
IVD.set_pseudo_bias_mode(0) # 0 (current bias) | 1 (voltage bias)
IVD.set_dAdV(2e-6)  # conversion factor of voltage biased current source (U -> I)
IVD.set_amp(1e3)  # preamplifier factor
IVD.set_BW(100)  # bandwidth of external filter (in Hz)
IVD.set_tau_amp(100)  # time constant of external amplifier (τ=2πRC in s)

In [ ]:
################
# voltage bias #
################
IVD.set_pseudo_bias_mode(1)  # voltage bias instead of IVD.set_pseudo_bias_mode(0) 
IVD.set_Vdiv(1e3)  # external voltage divider factor (e.g. 1e3)
IVD.set_dVdA(1e8)  # amplification factor for voltage bias (e.g. 1e9)

transport settings

In [ ]:
tr.set_dVdI(True)  # data series of numerical gradient
tr.set_average(2)  # average entire traces
tr.sweeps.reset_sweeps()
tr.add_sweep_4quadrants(start=0, stop=0, step=0, offset=0) # 4 quadrants sweep: start at zero -> stop -> zero -> -stop -> zero
#tr.sweeps.add_sweep(start=0, stop=0, step=0) # single sweep

2D scan settings (example: microwave power scan)

In [ ]:
mw_src.set_frequency(0.)  # set frequency
mw_src.set_status(True)

# x function that is executed at each iteration
def x_function(i):
    mw_src.enable_high_power = False
    mw_src.set_power(i)
    time.sleep(0.01)
    return

tr.set_x_parameters(x_vec=numpy.arange(0,0,0), # (start,stop,step) 
                    x_coordname='power', 
                    x_set_obj=x_function, 
                    x_unit ='dBm')

landscape scan (optional)

In [ ]:
def lsc_func(x, y0, y1, x0):
    y = np.ones(shape=len(x))*y1
    y[np.abs(x) > x0] = y0
    return y
tr.set_landscape(lsc_func, (0, 0, 0))

import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots()
ax.set_xlabel('{:s} ({:s})'.format(tr._x_coordname, tr._x_unit))
ax.set_ylabel('{:s} ({:s})'.format(tr._x_name, tr._x_unit))
ax.plot(tr._x_vec, tr._lsc_vec)
plt.show()

log function, e.g. temperature tracking (optional)

In [ ]:
def get_T():
    return tip.get_T()

tr.set_log_function([get_T], ['temp'], ['K'], ['f'])

start 2D measurement

In [ ]:
tr.measure_2D()

# 3D scan

IVD settings

In [ ]:
IVD.set_sense_nplc(1)  # number of power line cycles
IVD.set_sense_average(1)  # average
IVD.set_bias_delay(15e-6) # for Yokogawa GS820 it should be >= 200e-6 to ensure correct data storage
IVD.set_sense_delay(15e-6)  # measurement delay

In [ ]:
################
# current bias #
################
IVD.set_sweep_mode(0) # 0 (VV-mode) | 1 (IV-mode) | 2 (VI-mode)
IVD.set_pseudo_bias_mode(0) # 0 (current bias) | 1 (voltage bias)
IVD.set_dAdV(2e-6)  # conversion factor of voltage biased current source (U -> I)
IVD.set_amp(1e3)  # preamplifier factor
IVD.set_BW(100)  # bandwidth of external filter (in Hz)
IVD.set_tau_amp(100)  # time constant of external amplifier (τ=2πRC in s)

In [ ]:
################
# voltage bias #
################
IVD.set_pseudo_bias_mode(1)  # voltage bias instead of IVD.set_pseudo_bias_mode(0) 
IVD.set_Vdiv(1e3)  # external voltage divider factor (e.g. 1e3)
IVD.set_dVdA(1e8)  # amplification factor for voltage bias (e.g. 1e9)

transport settings

In [ ]:
tr.set_dVdI(True)  # data series of numerical gradient
tr.set_average(2)  # average entire traces
tr.sweeps.reset_sweeps()
tr.add_sweep_4quadrants(start=0, stop=0, step=0, offset=0) # 4 quadrants sweep: start at zero -> stop -> zero -> -stop -> zero
#tr.sweeps.add_sweep(start=0, stop=0, step=0) # single sweep

3D scan settings (example: microwave power and frequency scan)

In [ ]:
mw_src.set_status(True)

'''mw power variation'''
def x_function(i):
    mw_src.enable_high_power = False
    mw_src.set_power(i)
    time.sleep(0.01)
    return

tr.set_x_parameters(x_vec=numpy.arange(0,0,0), # (start,stop,step) 
                    x_coordname='power', 
                    x_set_obj=x_function, 
                    x_unit ='dBm')

'''mw frequency variation'''
def y_function(i):
    mw_src.set_status(True)
    time.sleep(0.1)
    mw_src.set_frequency(i)
    return 

tr.set_y_parameters(y_vec=numpy.arange(0.0e9, 20.0e9, 1.0e9), #  (start, stop, step)        
                    y_coordname='mw_frequency', 
                    y_set_obj=y_function, 
                    y_unit ='Hz')


#log function, e.g. temperature tracking
def get_T():
    return tip.get_T()

tr.set_log_function([get_T], ['temp'], ['K'], ['f'])

start 3D measurement

In [ ]:
tr.measure_3D()

# measure trace

e.g. time trace of voltage and current

In [ ]:
if hasattr(tr, 'tmp'): del(tr.tmp)
def get_time():
    if not hasattr(tr, 'tmp'):
        tr.tmp = time.time()
    return time.time()-tr.tmp

tr.set_xy_parameters(x_name='time',
                     x_func=get_time,
                     x_vec=np.arange(0, 10, 0.1),
                     x_unit='s',
                     y_name=['voltage', 'current'],
                     y_func=[IVD.get_voltage, IVD.get_current],
                     y_unit=['V', 'A'],
                     x_dt=1e-3)

tr.set_views(view=[(1,2)])  # add view I(V)

In [ ]:
tr.measure_xy()

# stop qkit

In [ ]:
IVD.close()  # in particular necessary for Keithley 2636A

In [ ]:
exit()